# Tuberías
Una habilidad fundamental para implementar (e incluso probar) modelos complejos con preprocesamiento.

En este tutorial, aprenderá cómo utilizar canalizaciones para limpiar su código de modelado.

### Introducción
Las canalizaciones son una forma sencilla de mantener organizado el código de modelado y preprocesamiento de datos. Específicamente, una canalización agrupa pasos de preprocesamiento y modelado para que pueda utilizar todo el paquete como si fuera un solo paso.

Muchos científicos de datos combinan modelos sin canalizaciones, pero las canalizaciones tienen algunos beneficios importantes. Estos incluyen:

1. Código más limpio: la contabilidad de los datos en cada paso del preprocesamiento puede resultar complicada. Con una canalización, no necesitará realizar un seguimiento manual de sus datos de capacitación y validación en cada paso.

2. Menos errores: hay menos oportunidades de aplicar mal un paso u olvidar un paso de preprocesamiento.

3. Más fácil de producir: puede resultar sorprendentemente difícil hacer la transición de un modelo de un prototipo a algo que se pueda implementar a escala. No entraremos aquí en las muchas preocupaciones relacionadas, pero los oleoductos pueden ayudar.

4. Más opciones para la validación del modelo: verá un ejemplo en el siguiente tutorial, que cubre la validación cruzada.

### Ejemplo
Como en el tutorial anterior, trabajaremos con el conjunto de datos de Melbourne Housing.

No nos centraremos en el paso de carga de datos. En cambio, puedes imaginar que estás en un punto en el que ya tienes los datos de entrenamiento y validación en X_train, X_valid, y_train e y_valid.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
data = pd.read_csv('melb_data.csv')

# Separate target from predictors
y = data.Price
X = data.drop(['Price'], axis=1)

# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

Echamos un vistazo a los datos de entrenamiento con el método head() a continuación. Observe que los datos contienen datos categóricos y columnas con valores faltantes. ¡Con una tubería, es fácil lidiar con ambos!

In [2]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.8272,4217.0


Construimos el pipeline completo en tres pasos.

### Paso 1: definir los pasos de preprocesamiento
De manera similar a cómo una canalización agrupa los pasos de preprocesamiento y modelado, usamos la clase ColumnTransformer para agrupar diferentes pasos de preprocesamiento. El siguiente código:

. imputa valores faltantes en datos numéricos, y

. imputa valores faltantes y aplica una codificación one-hot a datos categóricos.

In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

### Paso 2: definir el modelo
A continuación, definimos un modelo de bosque aleatorio con la conocida clase RandomForestRegressor.

In [3]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

### Paso 3: crear y evaluar la canalización
Finalmente, usamos la clase Pipeline para definir una canalización que agrupa los pasos de preprocesamiento y modelado. Hay algunas cosas importantes a tener en cuenta:

. Con el pipeline, preprocesamos los datos de entrenamiento y ajustamos el modelo en una sola línea de código. (Por el contrario, sin una canalización, tenemos que realizar la imputación, la codificación one-hot y el entrenamiento del modelo en pasos separados. ¡Esto se vuelve especialmente complicado si tenemos que lidiar con variables numéricas y categóricas!)

. Con la canalización, suministramos las características no procesadas en X_valid al comando predict() y la canalización preprocesa automáticamente las características antes de generar predicciones. (Sin embargo, sin una canalización, debemos recordar preprocesar los datos de validación antes de hacer predicciones).

In [13]:
Regionname_dummies = pd.get_dummies(X_train['Regionname'])

,Eastern Metropolitan,Eastern Victoria,Northern Metropolitan,Northern Victoria,South-Eastern Metropolitan,Southern Metropolitan,Western Metropolitan,Western Victoria
12167,0,0,0,0,0,1,0,0
6524,0,0,0,0,0,0,1,0
8413,0,0,0,0,0,0,1,0
2919,0,0,1,0,0,0,0,0
6043,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...
13123,0,0,1,0,0,0,0,0
3264,1,0,0,0,0,0,0,0
9845,0,0,1,0,0,0,0,0
10799,0,0,1,0,0,0,0,0


In [8]:
from sklearn.metrics import mean_absolute_error,r2_score
from sklearn.preprocessing import StandardScaler

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline([('preprocessor', preprocessor),
                              ('model', model), 
                              ('scaler', StandardScaler())
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
r2_score1 = r2_score(y_valid, preds)
print('MAE:', score)
print('R2score:', r2_score1)

TypeError: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'RandomForestRegressor(random_state=0)' (type <class 'sklearn.ensemble._forest.RandomForestRegressor'>) doesn't

### Conclusión
Las canalizaciones son valiosas para limpiar el código de aprendizaje automático y evitar errores, y son especialmente útiles para flujos de trabajo con preprocesamiento de datos sofisticado.

# EJERCICIOS

Trabajará con datos del concurso de precios de vivienda para usuarios de Kaggle Learn.

Ejecute la siguiente celda de código sin cambios para cargar los conjuntos de entrenamiento y validación en X_train, X_valid, y_train e y_valid. El conjunto de prueba se carga en X_test.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
X_full = pd.read_csv('../input/train.csv', index_col='Id')
X_test_full = pd.read_csv('../input/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

In [ ]:
X_train.head()

La siguiente celda de código utiliza código del tutorial para preprocesar los datos y entrenar un modelo. Ejecute este código sin cambios.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
model = RandomForestRegressor(n_estimators=100, random_state=0)

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

# Preprocessing of training data, fit model 
clf.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = clf.predict(X_valid)

print('MAE:', mean_absolute_error(y_valid, preds))

    MAE: 17614.81993150685

El código arroja un valor de alrededor de 17862 para el error absoluto medio (MAE). En el siguiente paso, modificará el código para mejorarlo.

### Paso 1: mejorar el rendimiento
#### Parte A
¡Ahora es tu turno! En la celda de código siguiente, defina sus propios pasos de preprocesamiento y su modelo de bosque aleatorio. Complete los valores para las siguientes variables:

. numerical_transformer

. categorical_transformer

. model

Para aprobar esta parte del ejercicio, solo necesita definir pasos de preprocesamiento válidos y un modelo de bosque aleatorio.

In [ ]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')
 # Your code here

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
]) # Your code here

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
model = RandomForestRegressor(n_estimators=100, random_state=0)

#### Parte B
Ejecute la celda de código a continuación sin cambios.

Para pasar este paso, debe haber definido una tubería en la Parte A que logre un MAE más bajo que el código anterior. Le recomendamos que se tome su tiempo aquí y pruebe muchos enfoques diferentes para ver qué tan bajo puede obtener el MAE. (Si su código no se aprueba, modifique los pasos de preprocesamiento y el modelo en la Parte A).

In [ ]:
# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

    MAE: 17540.473424657535

### Paso 2: generar predicciones de prueba
Ahora utilizará su modelo entrenado para generar predicciones con los datos de prueba.

In [ ]:
# Preprocessing of test data, fit model
preds_test = my_pipeline.predict(X_test) # Your code here

Ejecute la siguiente celda de código sin cambios para guardar sus resultados en un archivo CSV que pueda enviarse directamente a la competencia.